In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('/opt/slh/icecube/')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1,0"
#os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["NCCL_P2P_DISABLE"] = "1"

In [2]:
import polars as pl
import pandas as pd
import gc
import os
import numpy as np
from icecube.fastai_fix import *
from tqdm.notebook import tqdm
from icecube.data_train import RandomChunkSampler,LenMatchBatchSampler,IceCubeCache, DeviceDataLoader
from icecube.loss import loss, loss_vms
from icecube.models import EncoderWithDirectionReconstructionV16

/opt/conda/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /opt/conda/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE
  warn(f"Failed to load image Python extension: {e}")


graphnet: INFO     2023-03-22 18:52:54 - get_logger - Writing log to logs/graphnet_20230322-185254.log


In [3]:
OUT = 'hb_global_graphlocal'
PATH = '../data/'

NUM_WORKERS = 24
SEED = 2023
bs = 1024
L = 192

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(SEED)
os.makedirs(OUT, exist_ok=True)

In [4]:
def WrapperAdamW(param_groups,**kwargs):
    return OptimWrapper(param_groups,torch.optim.AdamW)



In [5]:
fname = OUT

ds_train = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125)
ds_train_len = IceCubeCache(PATH, mode='train', L=L, reduce_size=0.125, mask_only=True)
sampler_train = RandomChunkSampler(ds_train_len, chunks=ds_train.chunks)
len_sampler_train = LenMatchBatchSampler(sampler_train, batch_size=bs, drop_last=True)
dl_train = DeviceDataLoader(torch.utils.data.DataLoader(ds_train, 
            batch_sampler=len_sampler_train, num_workers=4, persistent_workers=True))

ds_val = IceCubeCache(PATH, mode='eval', L=L)
ds_val_len = IceCubeCache(PATH, mode='eval', L=L, mask_only=True)
sampler_val = torch.utils.data.SequentialSampler(ds_val_len)
len_sampler_val = LenMatchBatchSampler(sampler_val, batch_size=bs, drop_last=False)
dl_val= DeviceDataLoader(torch.utils.data.DataLoader(ds_val, batch_sampler=len_sampler_val,
            num_workers=0))

data = DataLoaders(dl_train,dl_val)
model = EncoderWithDirectionReconstructionV16()
model = nn.DataParallel(model)
learn = Learner(data,
                model,  
                path = OUT, 
                loss_func=loss_vms,
                cbs=[GradientClip(3.0),
                     CSVLogger(),
                     SaveModelCallback(monitor='loss',comp=np.less,every_epoch=True),
                     GradientAccumulation(n_acc=4)],
                     metrics=[loss], 
                     opt_func=partial(WrapperAdamW,eps=1e-7)).to_fp16()




In [6]:
learn.fit_one_cycle(8, lr_max=5e-4, wd=0.05, pct_start=0.01)

epoch,train_loss,valid_loss,loss,time
0,1.551161,1.563307,1.035551,2:22:30
1,1.484866,1.493966,1.036243,2:22:42


/opt/slh/icecube/icecube/data_train.py:253: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  L = max(1,L // 16)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /tmp/ipykernel_912773/4120003152.py:1 in <module>                                                │
│                                                                                                  │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_912773/4120003152.py'                       │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/callback/schedule.py:119 in fit_one_cycle          │
│                                                                                                  │
│   116 │   lr_max = np.array([h['lr'] for h in self.opt.hypers])                                  │
│   117 │   scheds = {'lr': combined_cos(pct_start, lr_max/div, lr_max, lr_max/div_final),         │
│   118 │   │   │     'mom': combined_cos(pct_start, *(self.moms if moms is None else moms))}      │
│ ❱ 119 │   self.fit(n_epoch, cbs=ParamScheduler(scheds)+L(cbs), reset_opt=reset_opt, wd=wd, sta   │
│   120                                                                                            │
│   121 # %% ../../nbs/14_callback.schedule.ipynb 50                                               │
│   122 @patch                                                                                     │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/learner.py:256 in fit                              │
│                                                                                                  │
│   253 │   │   │   if wd is not None: self.opt.set_hypers(wd=wd)                                  │
│   254 │   │   │   self.opt.set_hypers(lr=self.lr if lr is None else lr)                          │
│   255 │   │   │   self.n_epoch = n_epoch                                                         │
│ ❱ 256 │   │   │   self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup   │
│   257 │                                                                                          │
│   258 │   def _end_cleanup(self): self.dl,self.xb,self.yb,self.pred,self.loss = None,(None,),(   │
│   259 │   def __enter__(self): self(_before_epoch); return self                                  │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/learner.py:193 in _with_events                     │
│                                                                                                  │
│   190 │   │   self.xb,self.yb = b[:i],b[i:]                                                      │
│   191 │                                                                                          │
│   192 │   def _with_events(self, f, event_type, ex, final=noop):                                 │
│ ❱ 193 │   │   try: self(f'before_{event_type}');  f()                                            │
│   194 │   │   except ex: self(f'after_cancel_{event_type}')                                      │
│   195 │   │   self(f'after_{event_type}');  final()                                              │
│   196                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/fastai/learner.py:245 in _do_fit                          │
│                                                                                                  │
│   242 │   def _do_fit(self):                                                                     │
│   243 │   │   for epoch in range(self.n_epoch):                                                  │
│   244 │   │   │   self.epoch=epoch                         

In [ ]:
learn.validate()

In [ ]:
#learn.save('TEMP')